# Build custom container for Vertex training

In [1]:
!pwd

/home/jupyter/tf_vertex_agents/02-perarm-features-bandit


## Load env config

* use the prefix from `00-env-setup`

In [2]:
PREFIX = 'mabv1'

In [3]:
# staging GCS
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

# GCS bucket and paths
BUCKET_NAME              = f'{PREFIX}-{PROJECT_ID}-bucket'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

config = !gsutil cat {BUCKET_URI}/config/notebook_env.py
print(config.n)
exec(config.n)


PROJECT_ID               = "hybrid-vertex"
PROJECT_NUM              = "934903580331"
LOCATION                 = "us-central1"

REGION                   = "us-central1"
BQ_LOCATION              = "US"
VPC_NETWORK_NAME         = "ucaip-haystack-vpc-network"

VERTEX_SA                = "934903580331-compute@developer.gserviceaccount.com"

PREFIX                   = "mabv1"
VERSION                  = "v1"

BUCKET_NAME              = "mabv1-hybrid-vertex-bucket"
BUCKET_URI               = "gs://mabv1-hybrid-vertex-bucket"
DATA_GCS_PREFIX          = "data"
DATA_PATH                = "gs://mabv1-hybrid-vertex-bucket/data"
VOCAB_SUBDIR             = "vocabs"
VOCAB_FILENAME           = "vocab_dict.pkl"

VPC_NETWORK_FULL         = "projects/934903580331/global/networks/ucaip-haystack-vpc-network"

BIGQUERY_DATASET_ID      = "hybrid-vertex.movielens_dataset_mabv1"
BIGQUERY_TABLE_ID        = "hybrid-vertex.movielens_dataset_mabv1.training_dataset"

REPO_DOCKER_PATH_PREFIX  = "src"
RL_SUB_DIR     

In [4]:
! gsutil ls $BUCKET_URI

gs://mabv1-hybrid-vertex-bucket/acc-paf-v2/
gs://mabv1-hybrid-vertex-bucket/aiplatform-custom-training-2023-07-13-22:02:47.458/
gs://mabv1-hybrid-vertex-bucket/aiplatform-custom-training-2023-07-14-12:09:06.643/
gs://mabv1-hybrid-vertex-bucket/aiplatform-custom-training-2023-07-14-12:46:07.509/
gs://mabv1-hybrid-vertex-bucket/aiplatform-custom-training-2023-07-14-13:48:08.938/
gs://mabv1-hybrid-vertex-bucket/aiplatform-custom-training-2023-07-17-18:35:00.675/
gs://mabv1-hybrid-vertex-bucket/aiplatform-custom-training-2023-07-17-18:50:42.146/
gs://mabv1-hybrid-vertex-bucket/archived/
gs://mabv1-hybrid-vertex-bucket/banditos-2/
gs://mabv1-hybrid-vertex-bucket/baseline-bandit-v1/
gs://mabv1-hybrid-vertex-bucket/config/
gs://mabv1-hybrid-vertex-bucket/custom-neural-bandits-a100/
gs://mabv1-hybrid-vertex-bucket/custom-neural-bandits-v1/
gs://mabv1-hybrid-vertex-bucket/data/
gs://mabv1-hybrid-vertex-bucket/linear-bandit-v1/
gs://mabv1-hybrid-vertex-bucket/mab-local-classy-v1/
gs://mabv1-hybr

## imports

In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Build Image

## Container Image Variables

In [6]:
print(f"DOCKERNAME        = {DOCKERNAME}")
print(f"REPOSITORY        = {REPOSITORY}")
print(f"IMAGE_NAME        = {IMAGE_NAME}")
print(f"REMOTE_IMAGE_NAME = {REMOTE_IMAGE_NAME}")

DOCKERNAME        = Dockerfile_perarm_feats
REPOSITORY        = rl-movielens-mabv1
IMAGE_NAME        = train-perarm-feats-v1
REMOTE_IMAGE_NAME = us-central1-docker.pkg.dev/hybrid-vertex/rl-movielens-mabv1/train-perarm-feats-v1


## Create Artifact Repository

If you don't have an existing artifact repository, create one using the gcloud command below

In [7]:
# ! gcloud artifacts repositories create $REPOSITORY --repository-format=docker --location=$LOCATION

## Local Docker build

Provide a name for your dockerfile and make sure you are authenticated

In [8]:
! gcloud auth configure-docker $REGION-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


### Create Dockerfile

In [9]:
import os

root_path = '..'
os.chdir(root_path)
os.getcwd()

'/home/jupyter/tf_vertex_agents'

### Create train image

* see [example Dockerfile for GPU](https://github.com/GoogleCloudPlatform/cloudml-samples/blob/main/pytorch/containers/quickstart/mnist/Dockerfile-gpu) jobs in Vertex AI
* see deep learning container [example here](https://cloud.google.com/deep-learning-containers/docs/derivative-container), and here for [available DL containers](https://cloud.google.com/deep-learning-containers/docs/choosing-container#versions)

In [10]:
gpu_profiling = True # True | False

print(f"gpu_profiling : {gpu_profiling}")

gpu_profiling : True


In [11]:
# TRAIN_BASE_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-11:latest'
# docker pull tensorflow/tensorflow:2.13.0-gpu

if gpu_profiling:
    # TRAIN_BASE_IMAGE = 'tensorflow/tensorflow:2.13.0-gpu'
    TRAIN_BASE_IMAGE = 'gcr.io/deeplearning-platform-release/tf2-gpu.2-12.py310'
    NVTOP_RUN = 'RUN apt update && apt -y install nvtop'
    # NVTOP_RUN = 'RUN apt-get update && apt-get -y install nvtop'
else:
    TRAIN_BASE_IMAGE = 'python:3.10'
    NVTOP_RUN = None
    
RUN_EXPORT = "RUN export PYTHONPATH=${PYTHONPATH}:${APP_HOME}/"
    
print(f"TRAIN_BASE_IMAGE : {TRAIN_BASE_IMAGE}")
print(f"NVTOP_RUN        : {NVTOP_RUN}")
print(f"RUN_EXPORT       : {RUN_EXPORT}")

TRAIN_BASE_IMAGE : gcr.io/deeplearning-platform-release/tf2-gpu.2-12.py310
NVTOP_RUN        : RUN apt update && apt -y install nvtop
RUN_EXPORT       : RUN export PYTHONPATH=${PYTHONPATH}:${APP_HOME}/


In [12]:
dockerfile = f'''
FROM {TRAIN_BASE_IMAGE}

ENV PYTHONUNBUFFERED True

ENV APP_HOME /workspace

WORKDIR $APP_HOME

COPY /requirements.txt $APP_HOME/requirements.txt

RUN pip install --upgrade pip
RUN pip install --no-cache-dir -r $APP_HOME/requirements.txt

RUN ls $APP_HOME

COPY src/perarm_features $APP_HOME/src/perarm_features
COPY src/per_arm_rl $APP_HOME/src/per_arm_rl

{NVTOP_RUN}

RUN ls $APP_HOME

{RUN_EXPORT}

# Sets up the entry point to invoke the task.
ENTRYPOINT ["python3", "-m", "src.perarm_features.task"]
'''
print(dockerfile)


FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-12.py310

ENV PYTHONUNBUFFERED True

ENV APP_HOME /workspace

WORKDIR $APP_HOME

COPY /requirements.txt $APP_HOME/requirements.txt

RUN pip install --upgrade pip
RUN pip install --no-cache-dir -r $APP_HOME/requirements.txt

RUN ls $APP_HOME

COPY src/perarm_features $APP_HOME/src/perarm_features
COPY src/per_arm_rl $APP_HOME/src/per_arm_rl

RUN apt update && apt -y install nvtop

RUN ls $APP_HOME

RUN export PYTHONPATH=${PYTHONPATH}:${APP_HOME}/

# Sets up the entry point to invoke the task.
ENTRYPOINT ["python3", "-m", "src.perarm_features.task"]



In [13]:
with open(f'{DOCKERNAME}', 'w') as f:
    f.write(dockerfile)

### Build Image Locally

In [14]:
!docker build -t $REMOTE_IMAGE_NAME -f $DOCKERNAME .

Sending build context to Docker daemon  57.82MB
Step 1/14 : FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-12.py310
 ---> 2524f8ab2f7e
Step 2/14 : ENV PYTHONUNBUFFERED True
 ---> Using cache
 ---> 78620b4bd1ba
Step 3/14 : ENV APP_HOME /workspace
 ---> Using cache
 ---> af6c2c2d0959
Step 4/14 : WORKDIR $APP_HOME
 ---> Using cache
 ---> 5295cba61bdd
Step 5/14 : COPY /requirements.txt $APP_HOME/requirements.txt
 ---> Using cache
 ---> 9bb4f721d629
Step 6/14 : RUN pip install --upgrade pip
 ---> Using cache
 ---> 69182a76aef7
Step 7/14 : RUN pip install --no-cache-dir -r $APP_HOME/requirements.txt
 ---> Using cache
 ---> c4e8869aba16
Step 8/14 : RUN ls $APP_HOME
 ---> Using cache
 ---> 9c7f8de78b7f
Step 9/14 : COPY src/perarm_features $APP_HOME/src/perarm_features
 ---> a947d6459e4c
Step 10/14 : COPY src/per_arm_rl $APP_HOME/src/per_arm_rl
 ---> c6aee222d53f
Step 11/14 : RUN apt update && apt -y install nvtop
 ---> Running in 8304b7a96b6c


Get:1 http://security.ubuntu.com/ubuntu jamm

### Push container to Registry

In [15]:
# ### push the container to registry
!docker push $REMOTE_IMAGE_NAME

Using default tag: latest
The push refers to repository [us-central1-docker.pkg.dev/hybrid-vertex/rl-movielens-mabv1/train-perarm-feats-v1]

f0c66012: Preparing 
99436e5c: Preparing 
b118db97: Preparing 
9ca10308: Preparing 
6d90e42a: Preparing 
b03d4a93: Preparing 
11a33a47: Preparing 
3b613946: Preparing 
7ec48755: Preparing 
9ff1f1f7: Preparing 
1fb4c0a9: Preparing 
78387552: Preparing 
7aaa2b94: Preparing 
f6acfb89: Preparing 
6b281212: Preparing 
9c238a16: Preparing 
6524d219: Preparing 
336bfcaa: Preparing 
bf0cc5a6: Preparing 
b6b1b66a: Preparing 
7db027f2: Preparing 
4587e03b: Preparing 
54b6913d: Preparing 
03d8f0fd: Preparing 
7cfb894b: Preparing 
91ee4880: Preparing 
99a10105: Preparing 
e763119e: Preparing 
8921f21c: Preparing 
21ef34cd: Preparing 
dc117f9b: Preparing 
bf18a086: Preparing 
c08beb30: Preparing 
64fa0a1e: Preparing 
cf47cbae: Preparing 
cdbb4b45: Preparing 
b2bc65e7: Preparing 
a6feba89: Preparing 
03d4a93: Waiting g 
af0c2ae0: Preparing 
73079fce: Preparing 

### GPU profiling

> enter these commands in the Vertex interactive terminal:

```bash
sudo apt update
sudo apt -y install nvtop
```

**Finished**